In [ ]:
%load_ext autoreload
%autoreload 2

import pandas as pd
import numpy as np
from tqdm import tqdm
import os.path
import seaborn as sns
import matplotlib.pyplot as plt
from os.path import join
import dvu
dvu.set_style()
csv_dir = join('../data', 'PECARN_Registry', 'C. CSV Datasets')

def load_df(prefix='DIAGNOSISICD10'):
    csv_files = [f for f in os.listdir(csv_dir) if f.startswith(prefix) and f.endswith('.CSV')]
    dfs = []
    for f in tqdm(csv_files):
        dfs.append(pd.read_csv(join(csv_dir, f), engine='pyarrow'))
    df = pd.concat(dfs, ignore_index=True)
    return df

**Our background**
- The original PECARN analysis was based on 12,044 patients, 203 with IAI-I	([Holmes et al. 2013](https://pubmed.ncbi.nlm.nih.gov/23375510/).)
- In our stress-testing, we evaluate external validity of the PECARN IAI-I rule using the PSRC cohort of 2,188 patients, 62 with IAI-I ([Kornblith et al. 2022](https://journals.plos.org/digitalhealth/article?id=10.1371/journal.pdig.0000076))
- We also performed analysis in evaluating patient perspectives with LLMs for PECARN's PediDOSE EFIC trial ([Kornblith et al. 2025](https://www.nature.com/articles/s41598-025-89996-w))

**PECARN registry relevant patient counts**
- PECARN public registry includes 5,831,284 records across 2012-2021
- There is a fair amount of data for diagnoses relevant to IAI. The table below shows the number of records in the data corresponding to relevant codes in ICD-10 (and ICD-9 codes for data before 2016)

| S36 | S37 | S39.0 | S39.6 | S39.8 | S39.9 | 860 | 861 | 862 | 863 | 864 | 865 | 866 | 867 | 868 | 869 |
|-----|-----|-------|-------|-------|-------|-----|-----|-----|-----|-----|-----|-----|-----|-----|-----|
|5650 |2026 |4685   |0      |2318   |8663   |290  |378  |63   |322  |389  |383  |211  |82   |273  |10   |

Aggregating across these codes, we find a total of 16,647 unique patient visits corresponding to 16,391 unique patient IDs.

These patients received a large amount of imaging of different types, with total counts aggregated over the 16,647 unique patients as follows (note that patients often received multiple imaging types):

| Imaging type   |   Count |
|:--------------|--------:|
| XRAY          |   47344 |
| CT            |   11115 |
| Ultrasound    |    8735 |
| Other         |    8497 |
| MRI           |    1048 |

**PECARN registry linking patient counts to predictor variables**
- We evaluate the broad set of IAI-related patient visits above based on the predictor variables available in the PECARN registry
  - GCS-Score
    - We find that the among the set, 5.20% of patients have a GCS-Score of 14 or less, compared to 3.50% in the rest of the cohort
    - When restricting only to the ICD-10 codes, we find that 12.50% have a GCS-Score of 14 or less, compared to the 3.53% in the rest of the cohort
  - Pain-Score
    - We find that the among the set, patients have a mean pain score of 1.91, compared to 1.72 in the rest of the cohort
    - When restricting only to the ICD-10 codes, patients have a mean pain score of 1.99, compared to 1.72 in the rest of the cohort
- We would like to be able to link these patients to sufficient predictor variables / free text to be able to build and assess different types of models

### Load ICD-10

In [ ]:
df_ICD_10 = load_df('DIAGNOSISICD10')
df_ICD_9 = load_df('DIAGNOSISICD9')

In [ ]:
# notes on ICD codes
# S36*
# S37*
# S39.0 – Injury of muscle and tendon of abdomen, lower back, and pelvis
# S39.6 – Injury of intra-abdominal organ(s) with pelvic organ(s)
# S39.8 – Other specified injuries of abdomen, lower back, and pelvis
# S39.9 – Unspecified injury of abdomen, lower back, and pelvis
ICD_10_LIST = ['S36', 'S37', 'S39.0', 'S39.6', 'S39.8', 'S39.9']
ICD_9_LIST = ['860', '861', '862', '863',
              '864', '865', '866', '867', '868', '869']

In [ ]:
visit_ids_relevant = []
person_ids_relevant = []
for k in ICD_10_LIST:  # , 'S35']:
    print(k, df_ICD_10['DXCode'].str.startswith(k).sum())
    visit_ids_relevant.append(
        df_ICD_10.loc[df_ICD_10['DXCode'].str.startswith(k), 'VisitID'].unique())
    person_ids_relevant.append(
        df_ICD_10.loc[df_ICD_10['DXCode'].str.startswith(k), 'PersonID'].unique())

for k in ICD_9_LIST:
    print(k, df_ICD_9['DXCode'].str.startswith(k).sum())
    visit_ids_relevant.append(
        df_ICD_9.loc[df_ICD_9['DXCode'].str.startswith(k), 'VisitID'].unique())
    person_ids_relevant.append(
        df_ICD_9.loc[df_ICD_9['DXCode'].str.startswith(k), 'PersonID'].unique())

In [ ]:
visit_ids_relevant = np.unique(np.concatenate(visit_ids_relevant))
patient_ids_relevant = np.unique(np.concatenate(person_ids_relevant))
len(visit_ids_relevant), len(patient_ids_relevant)

### Load GCSScore

In [ ]:
df_GCS = load_df('GCS')

In [ ]:
# find intersection between GCS and visits
rel_idxs = df_GCS['VisitID'].isin(visit_ids_relevant)
# plt.hist(df_GCS['GCSTotal'][rel_idxs], bins=20)
sns.histplot(df_GCS['GCSTotal'][rel_idxs], bins=20, stat='probability')
# plt.hist(df_GCS['GCSTotal'][~rel_idxs], bins=20)
sns.histplot(df_GCS['GCSTotal'][~rel_idxs], bins=20,
             stat='probability', color='red')
plt.title('GCS Distribution')

print((df_GCS[rel_idxs]['GCSTotal'] < 15).mean())
print((df_GCS[~rel_idxs]['GCSTotal'] < 15).mean())

### Load Painscores

In [ ]:
df_pain = load_df('PAIN')

In [ ]:
rel_idxs = df_pain['VisitID'].isin(visit_ids_relevant)
print(df_pain['PainScore'][rel_idxs].mean())
print(df_pain['PainScore'][~rel_idxs].mean())

### Imaging

In [ ]:
df_im = load_df('IMAGING')

In [ ]:
rel_idxs = df_im['VisitID'].isin(visit_ids_relevant)
vals = df_im[rel_idxs]['RadTestType'].value_counts().head(20)

print(vals.T.to_markdown())

### Medications

In [ ]:
df_med = load_df('MEDICATIONS')
vals = df_med['MedName'].value_counts()
for k in vals.index:
    k_l = str(k.lower())
    if 'prbc' in k_l or 'platelets' in k_l or 'blood' in k_l or 'ffp' in k_l:
        print(k, vals[k])
# Medications doesn't record whether blood was received (atleast not based on keywords prbc, platelets, blood, or ffp)
vals.head(40)